In [ ]:
cd ..

In [ ]:
from src.imports import *
from src.transformers import *
from src.helpers import *
from src.vars import *
from src.tools import *

In [ ]:
train = pd.read_csv("data/source/train.csv")

In [ ]:
test = pd.read_csv("data/source/test.csv")

In [ ]:
train

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.dtypes

In [ ]:
train.dtypes.value_counts()

In [ ]:
train.describe(include=np.number).round(2)

In [ ]:
dd = train.describe(exclude=np.number).round(2)
dd

In [ ]:
train.Transported = train.Transported.astype(int)
train

In [ ]:
X = train.drop("Transported", axis=1)
y = train["Transported"]

In [ ]:
dd = dd.T
dd

In [ ]:
DISPLAY = False
FRAC = 0.1

In [ ]:
if DISPLAY:
    sns.pairplot(train.sample(frac=FRAC), corner=True, hue="Transported")

In [ ]:
pipeline

In [ ]:
# pipeline = Pipeline(
#     [
#         ("column_selector", ColumnSelector()),
#         ("log_transformer", LogTransformer()),
#         ("preprocessor", ColumnTransformer([])),
#         ("sampler_1", RandomUnderSampler()),
#         ("imputer", KNNImputer()),
#         ("scaler", StandardScaler()),
#         ("sampler_2", RandomUnderSampler()),
#         ("estimator", LogisticRegression()),
#     ]
# )

In [ ]:
# param_grid = {
#     "log_transformer__threshold": [0.5, 1, 3, 5, 10, 7, 12],
#     "preprocessor__transformers": [t0, t1],
#     "sampler_1": [RandomUnderSampler(), "passthrough"],
#     "scaler": scaler_list,
#     "sampler_2": [RandomUnderSampler(), "passthrough"],
#     "estimator": [
#         LogisticRegression(),
#     ],
# }

In [ ]:
# param_grid = {
#     "estimator":[ RandomForestClassifier(),],
#     "log_transformer__threshold":[ 0.5, 1, 3,],
#     "preprocessor__transformers":[ t1,],
#     "sampler_1":[ "passthrough",],
#     "sampler_2":[ "passthrough",],
#     "scaler":[ "passthrough"],
# }

In [ ]:
scaler_list

In [ ]:
param_grid = {
    "log_transformer__threshold": [0.3, 0.5, 0.75],
    "preprocessor__transformers": [t1],
    "sampler_1": ["passthrough"],  #  RandomUnderSampler()
    "sampler_2": ["passthrough", RandomUnderSampler()],
    "scaler": scaler_list,
    "estimator": [
        RandomForestClassifier(max_depth=128, min_samples_split=4, n_estimators=1280)
    ],
}

In [ ]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=cv(n=10, test_size=0.2),
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=2,
    refit=True,
    
)

grid.fit(X, y)

res = resultize(grid)
res

In [ ]:
save_predict_all(grid)

In [ ]:
grid_2 = GridSearchCV(
    pipeline,
    param_grid,
    cv=10,
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=2,
    refit=True,
    
)

grid_2.fit(X, y)

res_2 = resultize(grid_2)
res_2

In [ ]:
RandomForestClassifier

In [ ]:
# grid_ = grid.best_estimator_.fit(X,y)

In [ ]:
grid.best_estimator_[:-2]

In [ ]:
_X = grid.best_estimator_[:-2].transform(X)
_X

In [ ]:
pipeline_2 = Pipeline(
    [
        ("sampler", RandomUnderSampler()),
        ("estimator", RandomForestClassifier()),
    ]
)


In [ ]:
from itertools import product


In [ ]:
param_grid_2 = {
    "estimator__n_estimators": [ 40, 80, 160, 320, 640, 1280],
    "estimator__max_depth": [None,  4, 8, 16, 32, 64, 128],
    "estimator__min_samples_split": [ None, 4,  8, 16, 32, 64, 129],
    "estimator__oob_score": [True, False],
    "estimator__criterion": ["gini" ], # "entropy", "log_loss"
}

In [ ]:
len(list(product(*param_grid_2.values())))

In [ ]:
grid_2 = HalvingGridSearchCV(
    pipeline_2,
    param_grid_2,
    cv=5,
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=1,
    refit=True,
    aggressive_elimination=True,
    
)
grid_2

In [ ]:
grid_2.fit(_X, y)

In [ ]:
grid_2.best_estimator_

In [ ]:
res_2 = resultize(grid_2)
res_2

In [ ]:
save_predict_all(grid)

In [ ]:
1/0

In [ ]:
param_grid = {
    "log_transformer__threshold": [3, 5, 10, 7, 12],
    "preprocessor__transformers": [t0, t1],
    "sampler_1": [RandomUnderSampler(), "passthrough"],
    "scaler": ["passthrough", StandardScaler(), Normalizer()],
    "sampler_2": ["passthrough"],
    "estimator": [
        RandomForestClassifier(),
    ],
}

In [ ]:
grid = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,  # StratifiedShuffleSplit(n_splits=10, test_size=0.2, random_state=42)
    scoring="accuracy",
    n_jobs=12,
    return_train_score=True,
    verbose=1,
)

grid.fit(X, y)

In [ ]:
res = resultize(grid).head(10)
res

In [ ]:
grid

In [ ]:
test

In [ ]:
train

In [ ]:
def find_bool(X) : 

    ser = X.iloc[0]


    print(ser)

    print(ser.dtypes)


find_bool(train)

In [ ]:
v = train.iloc[0].CryoSleep
v

In [ ]:
type(v)

In [ ]:
np.bool_

In [ ]:
X.CryoSleep.astype(int)

In [ ]:
bool_cols=["CryoSleep", "VIP"]

In [ ]:
_X = X.copy()

In [ ]:

for col in bool_cols : 
    _X[col] = _X[col].replace({True: 1, False: 0})

In [ ]:
_X

In [ ]:
split_cols=["Cabin"]

In [ ]:
for col in split_cols : 
    for i in range(2) : 
        _X[col+_i] = _X[col].apply(lambda i : i.split("/")[i])



In [ ]:
c = X.iloc[0].Cabin
c

In [ ]:
X.Cabin.isna().mean()

In [ ]:
for col in split_cols :
    for i in range(2) : 
        _X[col+_i] = _X[col].apply(lambda i : i.split("/")[i] if isinstance(i, str) else i  )


In [ ]:
for col in split_cols :
    for i in range(2) : 
        _X[col+f"_{i}"] = _X[col].apply(lambda i : i.split("/")[i] if isinstance(i, str) else i  )


In [ ]:
for col in split_cols :
    for i in range(3) : 
        _X[col+f"_{i}"] = _X[col].apply(lambda val : val.split("/")[i] if isinstance(val, str) else val  )
        try : 
            _X[col+f"_{i}"] = _X[col+f"_{i}"].astype(float)
        except Exception as e : 
            pass


In [ ]:
split_cols

In [ ]:
_X

In [ ]:
_X.Cabin_0.value_counts()

In [ ]:
_X.Cabin_1.value_counts()

In [ ]:
_X.Cabin_2.value_counts()

In [ ]:
_X.Name

In [ ]:
ser = _X.Name.apply(lambda i : len(i.split(" ")) if isinstance(i, str) else i)

In [ ]:
ser

In [ ]:
ser.value_counts()

In [ ]:
ser =  _X.Name.apply(lambda i : len(i) if isinstance(i, str) else i)


In [ ]:
ser

In [ ]:
            # _X.Name.apply(lambda i : len(i.split(" ")) if isinstance(i, str) else i) # no because only 2 on train
            _X["_len_Name"] = _X.Name.apply(lambda i : len(i) if isinstance(i, str) else i)
            _X["_FirstName"] = _X.Name.apply(lambda i : i.split(" ")[0] if isinstance(i, str) else i)
            _X["_LastName"] = _X.Name.apply(lambda i : i.split(" ")[1] if isinstance(i, str) else i)



In [ ]:
_X

In [ ]:
_X.isna().mean().round(2)

In [ ]:
tmp = _X.isna().mean(axis=1).round(2)
tmp

In [ ]:
tmp[tmp>0.0].sort_values()

In [ ]:
_X._LastName.value_counts().to_dict()

In [ ]:
[1] if isinstance(i, str) else i)
            family_dict = _X._LastName.value_counts().to_dict()
            _X["_FamilySize"] = _X["_LastName"].replace(family_dict

In [ ]:
family_dict = _X._LastName.value_counts().to_dict()
_X["_FamilySize"] = _X["_LastName"].replace(family_dict)

In [ ]:
_X.drop(columns=[], errors="ignore")

In [ ]:
_X.Cabin_0.unique()

In [ ]:
import string

In [ ]:
string.ascii_uppercase

In [ ]:
cab_dict = {i:j for j,i in enumerate(string.ascii_uppercase)}
cab_dict

In [ ]:
for col in split_cols :
    for i in range(3) : 
        _X[f"_{col}_{i}"] = _X[col].apply(lambda val : val.split("/")[i].upper() if isinstance(val, str) else val  )
        try : 
            _X[f"_{col}_{i}"] = _X[f"_{col}_{i}"].astype(float)
        except Exception as e : 
            _X[f"_{col}_{i}_int"] =  _X[f"_{col}_{i}"].replace(cab_dict)


In [ ]:
_X.iloc[:,  3:]